# CSP
**通信顺序进程，Communicating Sequential Process**

- Tony Hoare, 1977
- A concurrent programming language
- A process calculi/algebra

- 不关注发送消息的**process（进程/主体）**
- 关注发送消息时使用的**channel（通道）**

- process类似于Actor模型中的Actor，channel类似于Mailbox
- channel是一级对象，不与process紧耦合，而Mailbox是绑定至Actor的
- channel可以单独创建和读写，并在process间传递
- process可以没有实体，可以订阅任意个channel或向任意个channel发送消息

# Go语言中的Goroutine和Channel

- 消息通过channel在goroutine之间传递
- goroutine是process，是主体
- channel是通道

```Go
f() // call f(); wait for it to return
go f() // create a new goroutine that calls f(); don't wait
```

```Go
ch := make(chan int) // 创建channel，收发int类型消息

ch <- x // 发送消息
x = <-ch // 接收并赋值消息
<-ch // 接收但丢弃消息

close(ch) // 关闭channel
```

```Go
ch = make(chan int) // 无缓冲channel
ch = make(chan int, 0) // 无缓冲channel
ch = make(chan int, 3) // 缓冲channel，容量为3
```

**无缓冲channel**，也称为**同步channel**
> 在无缓冲channel上的一次发送和接收操作
>
> 导致发送和接收goroutine做了一次同步操作
>
> 接收者收到数据发生在唤醒发送者goroutine之前（happens before）

**缓冲channel**，可以看作是**消息队列**

![](empty-buffered-channel.png)

In [14]:
import (
    "net/http"
    "time"
)

request := func(url string) *http.Response {
    rep, _ := http.Get(url)
    return rep
}

responses := make(chan *http.Response, 3)
go func() { responses <- request("https://www.baidu.com") }()
go func() { responses <- request("https://www.taobao.com") }()
go func() { responses <- request("http://www.qq.com") }()
go func() { time.Sleep(100*time.Millisecond); close(responses); }()

for r := range responses {
    fmt.Printf("%s: %s\n", r.Request.URL, r.Status)
}

http://www.qq.com: 200 OK
https://www.taobao.com: 200 OK
https://www.baidu.com: 200 OK


## 串联channel（pipeline）

![](three-stage-pipeline.png)

In [1]:
naturals := make(chan int)
squares := make(chan int)

// Counter
go func() {
    for x := 0; x < 10; x++ {
        naturals <- x
    }
    close(naturals)
}()

// Squarer
go func() {
    for x := range naturals {
        squares <- x * x
    }
    close(squares)
}()

// Printer (in main goroutine)
for x := range squares {
    fmt.Printf("%d ", x)
}

0 1 4 9 16 25 36 49 64 81 

## 单向channel
- 只能发送：```chan<- int```
- 只能接收：```<-chan int```

In [2]:
counter := func(out chan<- int) {
    for x := 0; x < 10; x++ {
        out <- x
    }
    close(out)
}

squarer := func(out chan<- int, in <-chan int) {
    for v := range in {
        out <- v * v
    }
    close(out)
}

printer := func(in <-chan int) {
    for v := range in {
        fmt.Printf("%d ", v)
    }
}

naturals := make(chan int)
squares := make(chan int)
go counter(naturals)
go squarer(squares, naturals)
printer(squares)

0 1 4 9 16 25 36 49 64 81 

## 多路复用：select

```Go
select {
case <-ch1: // 接收并丢弃
// ...
case x := <-ch2: // 接收
// ...use x...
case ch3 <- y: // 发送
// ...
default: // 不阻塞，可以用于实现轮询channel
// ...
}
```

一发一收，依次执行

In [57]:
ch := make(chan int, 1)
for i := 0; i < 10; i++ {
    select {
    case x := <-ch:
        fmt.Printf("%d ", x) // "0" "2" "4" "6" "8"
    case ch <- i:
    }
}

0 2 4 6 8 

当多个分支同时满足时，**随机**选择一个分支执行

In [65]:
ch := make(chan int, 2)
for i := 0; i < 10; i++ {
    select {
    case x := <-ch:
        fmt.Printf("%d ", x) // "0" "2" "4" "6" "8"
    case ch <- i:
    }
}

0 2 3 6 

## 并发退出：关闭channel

In [1]:
import "time"

ch := make(chan int)

go func() {
    ch <- 1
    ch <- 2
}()

go func() {
    ch <- 3
    ch <- 4
}()

go func() {
    time.Sleep(10*time.Millisecond)
    close(ch)
}()

for {
    select {
    case x := <-ch:
        if x == 0 {
            fmt.Printf("\nBye")
            return
        }
        fmt.Printf("%d ", x)
    }
}

1 2 3 4 
Bye